# Create VIEWs on SEC DERA data by LEI

## Full-year FY REVENUES, INCOME

## Point-in-time FLOAT, DEBT, CASH, ASSETS, (EV = FLOAT + DEBT - CASH, EVIC = FLOAT + DEBT)

In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [2]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

In [3]:
from osc_ingest_trino import *

trino_bucket = attach_s3_bucket("S3_DEV")

In [4]:
for tbl in ['t_r', 't_f', 't_d', 't_c', 't_a']:
    qres = engine.execute(f"drop table if exists sec_dera.{tbl}")
    print(qres.fetchall())

[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]


### FX

Many public companies are non-US companies or report facts in units of measurement other than USD.  We build a small table of market knowledge so that we can do foreign exchange (FX) calculations.

In [5]:
sql = ("""
select count (*) as count, uom, ddate, max(value) as maxval, min(value) as minval from sec_dera.num
where ddate>=DATE('2019-01-01')
      and (tag='ClosingForeignExchangeRate' or tag='ForeignCurrencyExchangeRateTranslation1')
      and qtrs=0 and uom!= 'USD' and (uom like '%USD%' or length(uom)=3)
group by uom, ddate, tag
order by ddate, uom, tag
""")
df_closing_fx = pd.read_sql (sql, engine, parse_dates=['ddate'])
# display(df_closing_fx)

sql = ("""
select count (*) as count, uom, ddate, max(value) as maxval, min(value) as minval from sec_dera.num
where ddate>=DATE('2019-01-01')
      and tag='AverageForeignExchangeRate'
      and qtrs=4 and uom!= 'USD' and (uom like '%USD%' or length(uom)=3)
group by uom, ddate, tag
order by ddate, uom, tag
""")
df_average_fx = pd.read_sql (sql, engine, parse_dates=['ddate'])
# display(df_average_fx)

In [6]:
import re

def create_fx_dict(df):
    """From a list of FX pairs or non-USD (presumed to convert to USD) FX rates by dates, return a dictionary of FX rates by dates"""
    df_gbp = df[df.uom.str.contains('GBP', flags=re.I)]
    df_eur = df[df.uom.str.contains('EUR', flags=re.I)]
    df_other = df[~df.uom.str.contains('(GBP)|(EUR)', flags=re.I)]
    
    fx_input = {'GBP': df_gbp, 'EUR': df_eur }

    fx_df = {}

    for fx in ['GBP', 'EUR']:
        df_base = fx_input[fx].loc[df.maxval>1, ['ddate', 'maxval']].groupby(by='ddate').mean()
        df_base['minval'] = 1/df_base.maxval
        df_quote = fx_input[fx].loc[df.minval<1, ['ddate', 'minval']].groupby(by='ddate').mean()
        df_quote['maxval'] = 1/df_quote.minval
        df_fx = pd.concat((df_base, df_quote), axis=0).groupby(by='ddate').mean().reset_index()
        df_fx['base'] = fx
        df_fx.rename(columns={'maxval':'to_USD', 'minval':'from_USD'}, inplace=True)
        df_fx = df_fx[['base', 'to_USD', 'from_USD', 'ddate']]
        fx_df[fx] = df_fx.convert_dtypes()
    
    for fx in list({x[int(x[-1]=='USD')-1].upper():x[int(x[0]=='USD')-1].upper() for x in df_other.uom.str.split(r'[-_/]').values }.keys()):
        df_other_fx = df_other[df_other.uom.str.contains(fx, flags=re.I)]
        df_base = df_other_fx.loc[df.maxval>1, ['ddate', 'maxval']].groupby(by='ddate').mean()
        df_base['minval'] = 1/df_base.maxval
        df_quote = df_other_fx.loc[df.minval<1, ['ddate', 'minval']].groupby(by='ddate').mean()
        df_quote['maxval'] = 1/df_quote.minval
        df_fx = pd.concat((df_base, df_quote), axis=0).groupby(by='ddate').mean().reset_index()
        df_fx['base'] = fx
        df_fx.rename(columns={'maxval':'from_USD', 'minval':'to_USD'}, inplace=True)
        df_fx = df_fx[['base', 'to_USD', 'from_USD', 'ddate']]
        fx_df[fx] = df_fx.convert_dtypes()
    
    return fx_df

The `closing_fx` table contains point-in-time FX market information.

The `average_fx` table contains a four-quarter average of FX rates.

In [7]:
import io

ingest_schema = 'sec_dera'

def create_fx_table(df_fx, tbl):
    qres = engine.execute(f"drop table if exists {ingest_schema}.{tbl}")
    display(qres.fetchall())

    df = enforce_sql_column_names(pd.concat(create_fx_dict(df_fx).values()))

    buf = io.BytesIO()
    df.to_parquet(path=buf)
    buf.seek(0)
    trino_bucket.upload_fileobj(Fileobj=buf,
                                Key=f'trino/{ingest_schema}/{tbl}/data.parquet')

    columnschema = create_table_schema_pairs(df, typemap={'datetime64[ns]':'timestamp(3)'})
    tabledef = f"""
create table if not exists {ingest_schema}.{tbl} (
{columnschema}
) with (
format = 'parquet',
external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{tbl}/'
)
"""
    print(tabledef)

    table_create = engine.execute(tabledef)
    # display(table_create.fetchall())

    if False:
        dataset_query = (f'SELECT * FROM {ingest_schema}.{tbl} limit 10')
        print(dataset_query)
        dataset = engine.execute(dataset_query)
        display(dataset.fetchall())

for df, tbl in [ (df_closing_fx,'closing_fx'), (df_average_fx,'average_fx') ]:
    create_fx_table(df, tbl)

[(True,)]

<ipython-input-6-e343598a9137>:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_other = df[~df.uom.str.contains('(GBP)|(EUR)', flags=re.I)]



create table if not exists sec_dera.closing_fx (
    base varchar,
    to_usd double,
    from_usd double,
    ddate timestamp(3)
) with (
format = 'parquet',
external_location = 's3a://ocp-odh-os-demo-s3/trino/sec_dera/closing_fx/'
)



[(True,)]


create table if not exists sec_dera.average_fx (
    base varchar,
    to_usd double,
    from_usd double,
    ddate timestamp(3)
) with (
format = 'parquet',
external_location = 's3a://ocp-odh-os-demo-s3/trino/sec_dera/average_fx/'
)



### Revenue

For this table and others, we do not yet implement a mechanism for prioritizing amended values over initially reported values.  That is a TODO.

engine.execute("select *, if (fp='','fp should not be empty string', '') from sec_dera.sub as S where S.form='10-K' and S.fp!='FY'").fetchall()

In [8]:
qres = engine.execute("drop table if exists sec_dera.t_r")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.fy_revenue_usd_by_lei as
select S.adsh, cik, name, lei, sic, fy, N.ddate, uom, max(value) as revenue
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and uom='USD'
        and coreg is NULL
        and qtrs=4
        and (N.tag='Revenue'
             or N.tag='Revenues'
             or N.tag='RevenueFromContractsWithCustomers'
             or N.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
             or N.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
             or N.tag='RevenuesNetOfInterestExpense'
             or N.tag='RegulatedAndUnregulatedOperatingRevenue'
             or N.tag='RegulatedOperatingRevenuePipelines')
group by S.adsh, cik, name, lei, sic, fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.fy_revenue_xyz_by_lei as
select S.adsh, S.cik, S.name, S.lei, S.sic, S.fy, N.ddate, N.uom, max(N.value) as revenue
from sec_dera.sub as S left join sec_dera.fy_revenue_usd_by_lei X on S.adsh=X.adsh
     join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate) and X.adsh IS NULL
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and N.uom!='USD'
        and coreg is NULL
        and qtrs=4
        and (N.tag='Revenue'
             or N.tag='Revenues'
             or N.tag='RevenueFromContractsWithCustomers'
             or N.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
             or N.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
             or N.tag='RevenuesNetOfInterestExpense'
             or N.tag='RegulatedAndUnregulatedOperatingRevenue'
             or N.tag='RegulatedOperatingRevenuePipelines')
group by S.adsh, S.cik, S.name, S.lei, S.sic, S.fy, N.ddate, N.uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_r as
select V_USD.*, V_USD.revenue as revenue_usd
from sec_dera.fy_revenue_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.revenue as revenue_usd
from sec_dera.fy_revenue_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(True,)]

[(11162,)]

### Float (market cap)

TODO: The float calculation reads the reported overall float without regard to share class.  For companies that have multiple associated tickers, this gives wrong results.  And probably very wrong results for BRK.A vs. BRK.B

In [9]:
qres = engine.execute("drop table if exists sec_dera.t_f").fetchall()
qres = engine.execute("""
create or replace view sec_dera.float_by_adsh_ddate as
select S.adsh, fy, max(ddate) as f_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and coreg is NULL
        and qtrs=0
        and (tag='EntityPublicFloat'
             or tag='EntitysPublicFloat'
             or tag='FreeFloat'
             or tag='PublicFloat'
             or tag='PublicFloatValue')
group by S.adsh, fy
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.float_usd_by_lei as
select S.adsh, S.cik, name, lei, tname, sic, S.fy, N.ddate, uom, max(value) as market_cap
from sec_dera.sub as S
     join sec_dera.float_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.f_ddate=N.ddate
     left join sec_dera.ticker T on S.cik=T.cik
     -- we already know we are looking at the correct S.form type
where value>0
      and uom='USD'
      and coreg is NULL
      and qtrs=0
      and (tag='EntityPublicFloat'
           or tag='EntitysPublicFloat'
           or tag='FreeFloat'
           or tag='PublicFloat'
           or tag='PublicFloatValue')
group by S.adsh, S.cik, name, lei, tname, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.float_xyz_by_lei as
select S.adsh, S.cik, name, lei, tname, sic, S.fy, N.ddate, uom, max(value) as market_cap
from sec_dera.sub as S
     join sec_dera.float_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.f_ddate=N.ddate
     left join sec_dera.ticker T on S.cik=T.cik
     -- we already know we are looking at the correct S.form type
where value>0
      and uom!='USD'
      and coreg is NULL
      and qtrs=0
      and (tag='EntityPublicFloat'
           or tag='EntitysPublicFloat'
           or tag='FreeFloat'
           or tag='PublicFloat'
           or tag='PublicFloatValue')
group by S.adsh, S.cik, name, lei, tname, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_f as
select V_USD.*, V_USD.market_cap as market_cap_usd
from sec_dera.float_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.market_cap as market_cap_usd
from sec_dera.float_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]

[(True,)]

[(True,)]

[(11170,)]

### Cash

In [10]:
qres = engine.execute("drop table if exists sec_dera.t_c").fetchall()
qres = engine.execute("""
create or replace view sec_dera.cash_by_adsh_ddate as
select S.adsh, fy, max(ddate) as c_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and coreg is NULL
        and qtrs=0
        and (tag='Cash'
             or tag='CashAndDueFromBanks'
             or tag='CashAndCashEquivalents'
             or lower(tag)=lower('CashAndCashEquivalentsUnrestricted')
             or tag='CashEquivalentsAtCarryingValue' 
             or tag='CashAndCashEquivalentsAtCarryingValue'
             or tag='CashAndCashEquivalentsAtCarryingValueExcludingVariableInterestEntities')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.cash_usd_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as cash
from sec_dera.sub as S
     join sec_dera.cash_by_adsh_ddate as AD on S.adsh=AD.adsh -- and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.c_ddate=N.ddate
where coreg is NULL
      and uom='USD'
      and qtrs=0
      and (tag='Cash'
             or tag='CashAndDueFromBanks'
             or tag='CashAndCashEquivalents'
             or lower(tag)=lower('CashAndCashEquivalentsUnrestricted')
             or tag='CashEquivalentsAtCarryingValue' 
             or tag='CashAndCashEquivalentsAtCarryingValue'
             or tag='CashAndCashEquivalentsAtCarryingValueExcludingVariableInterestEntities')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.cash_xyz_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as cash
from sec_dera.sub as S
     join sec_dera.cash_by_adsh_ddate as AD on S.adsh=AD.adsh -- and S.fy=AD.fy
     join sec_dera.num as N on AD.adsh=N.adsh and AD.c_ddate=N.ddate
where coreg is NULL
      and uom!='USD'
      and qtrs=0
      and (tag='Cash'
             or tag='CashAndDueFromBanks'
             or tag='CashAndCashEquivalents'
             or lower(tag)=lower('CashAndCashEquivalentsUnrestricted')
             or tag='CashEquivalentsAtCarryingValue' 
             or tag='CashAndCashEquivalentsAtCarryingValue'
             or tag='CashAndCashEquivalentsAtCarryingValueExcludingVariableInterestEntities')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_c as
select V_USD.*, V_USD.cash as cash_usd
from sec_dera.cash_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.cash as cash_usd
from sec_dera.cash_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(True,)]

[(13224,)]

### Debt

In [11]:
# Reiterating comment embedded below: we should sum 'LongTermDebtNoncurrent' and 'LongTermDebtCurrent' as one term we MAX with others
qres = engine.execute("drop table if exists sec_dera.t_d").fetchall()

qres = engine.execute("""
create or replace view sec_dera.debt_by_adsh_ddate as
select S.adsh, fy, max(ddate) as d_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and coreg is NULL
        and qtrs=0
        and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
             or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
             or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
             or tag='OperatingLeaseLiabilityNoncurrent'
             or tag='SubordinatedDebt' or tag='ConvertibleDebt'
             or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
             -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
             or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by S.adsh, fy
""")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.debt_usd_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as debt
from sec_dera.sub as S
     join sec_dera.debt_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom='USD'
      and qtrs=0
      and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
           or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
           or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
           or tag='OperatingLeaseLiabilityNoncurrent'
           or tag='SubordinatedDebt' or tag='ConvertibleDebt'
           or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
           -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
           or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.debt_xyz_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as debt
from sec_dera.sub as S
     join sec_dera.debt_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom!='USD'
      and qtrs=0
      and (tag='LongTermDebt' or tag='LongTermDebtFairValue'
           or tag='LongTermDebtAndCapitalLeaseObligations' or tag='DebtAndCapitalLeaseObligations'
           or tag='DebtLongtermAndShorttermCombinedAmount' or tag='SecuredDebt' or tag='UnsecuredDebt'
           or tag='OperatingLeaseLiabilityNoncurrent'
           or tag='SubordinatedDebt' or tag='ConvertibleDebt'
           or tag='LongTermLineOfCredit' or tag='OtherBorrowings' or tag='NotesAndLoansReceivableNetNoncurrent'
           -- NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
           or tag='LongTermDebtNoncurrent' or tag='LongTermDebtCurrent')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_d as
select V_USD.*, V_USD.debt as debt_usd
from sec_dera.debt_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.debt as debt_usd
from sec_dera.debt_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(True,)]

[(9441,)]

### Assets

In [12]:
qres = engine.execute("drop table if exists sec_dera.t_a")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.assets_by_adsh_ddate as
select S.adsh, fy, max(ddate) as d_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and coreg is NULL
        and qtrs=0
        and tag='Assets'
group by S.adsh, fy
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.assets_usd_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as assets
from sec_dera.sub as S
     join sec_dera.assets_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom='USD'
      and qtrs=0
      and tag='Assets'
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.assets_xyz_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as assets
from sec_dera.sub as S
     join sec_dera.assets_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom!='USD'
      and qtrs=0
      and tag='Assets'
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_a as
select V_USD.*, V_USD.assets as assets_usd
from sec_dera.assets_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.assets as assets_usd
from sec_dera.assets_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]


[(True,)]

[(True,)]

[(True,)]

[(13484,)]

### Income (Annual Profit / Loss)

In [13]:
qres = engine.execute("drop table if exists sec_dera.t_i")
print(qres.fetchall())
qres = engine.execute("drop view if exists sec_dera.sec_dera.income_usd_by_lei")
print(qres.fetchall())
qres = engine.execute("drop view if exists sec_dera.sec_dera.income_xyz_by_lei")
print(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.fy_income_by_adsh_ddate as
select S.adsh, fy, max(ddate) as d_ddate
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh and year(S.fy)=year(N.ddate)
where (S.form='10-K' or S.form='20-F' or S.form='40-F')
        and S.fy>=DATE('2019-01-01')
        and coreg is NULL
        and qtrs=4
        and (tag='ProfitLoss'
             or tag='NetIncomeLoss')
group by S.adsh, fy
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.fy_income_usd_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as income
from sec_dera.sub as S
     join sec_dera.fy_income_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom='USD'
      and qtrs=4
      and (tag='ProfitLoss'
             or tag='NetIncomeLoss')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create or replace view sec_dera.fy_income_xyz_by_lei as
select S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom, max(value) as income
from sec_dera.sub as S
     join sec_dera.fy_income_by_adsh_ddate as AD on S.adsh=AD.adsh
     join sec_dera.num as N on AD.adsh=N.adsh and AD.d_ddate=N.ddate
where coreg is NULL
      and uom!='USD'
      and qtrs=4
      and (tag='ProfitLoss'
             or tag='NetIncomeLoss')
group by S.adsh, S.cik, name, lei, sic, S.fy, N.ddate, uom
""")
display(qres.fetchall())

qres = engine.execute("""
create table sec_dera.t_i as
select V_USD.*, V_USD.income as income_usd
from sec_dera.fy_income_usd_by_lei as V_USD
UNION ALL
select V_XYZ.*, coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.income as income_usd
from sec_dera.fy_income_xyz_by_lei as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     left join sec_dera.closing_fx FX_1mo on V_XYZ.uom=FX_1mo.base and date_diff('month', FX_1mo.ddate, V_XYZ.ddate)=1
     left join sec_dera.closing_fx FX_2mo on V_XYZ.uom=FX_2mo.base and date_diff('month', FX_2mo.ddate, V_XYZ.ddate)=2
     left join sec_dera.closing_fx FX_3mo on V_XYZ.uom=FX_3mo.base and date_diff('month', FX_3mo.ddate, V_XYZ.ddate)=3
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
""")
display(qres.fetchall())

[(True,)]
[(True,)]
[(True,)]


[(True,)]

[(True,)]

[(True,)]

[(13142,)]

In [14]:
qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select R.name, R.lei,
       F.tname,
       R.fy, R.ddate, R.sic, revenue_usd, cash_usd, debt_usd, assets_usd, market_cap_usd, income_usd
from sec_dera.t_r as R
     left join sec_dera.t_c as C on R.cik=C.cik and R.adsh=C.adsh
     left join sec_dera.t_d as D on R.cik=D.cik and R.adsh=D.adsh
     left join sec_dera.t_a as A on R.cik=A.cik and R.adsh=A.adsh
     left join sec_dera.t_f as F on R.cik=F.cik and R.adsh=F.adsh
     left join sec_dera.t_i as I on R.cik=I.cik and R.adsh=I.adsh
where tname is null or tname not like '%-%'
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei
""")
l = qres.fetchall()
len(l)

[(True,)]

14657

In [15]:
y = [x for x in l if x[5] == 4911]
print(len(y))
display(y)

157


[('ELLOMAY CAPITAL LTD.', '254900FHSYEEYLHR7Y05', None, '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 4911, 11841313.188075509, 82066623.12668817, None, 564960163.0257214, None, -7572547.4073664835),
 ('ENERGY CO OF MINAS GERAIS', None, None, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', 4911, 5941683047.833006, 125432930.82467471, None, 11683515866.329681, None, 732004118.6932509),
 ('AZURE POWER GLOBAL LTD', '54930085G5UQETD7JX48', None, '2019-01-01 00:00:00.000', '2019-03-31 00:00:00.000', 4911, 143228333.60323888, 152059256.44881436, 994878492.53904, 1570865334.4129555, None, 1991286.5240023136),
 ('EDENOR', '254900JU7TV6VS9GRW29', None, '2019-01-01 00:00:00.000', '2019-12-31 00:00:00.000', 4911, 1501816563.7001169, 6839906.495241276, None, 1994867406.9126732, None, 202607096.34329605),
 ('CENTRAL PUERTO S.A.', '2549005JLIWQ6B7VNR38', None, '2020-01-01 00:00:00.000', '2020-12-31 00:00:00.000', 4911, 452859893.04812837, 3311919.1919191917, None, 1882229447.4153297, N

In [16]:
l = engine.execute("""
select A.lei, A.parent_name, R.name, date_format(R.fy, '%Y-%m-%d'), R.revenue_usd
from (select coalesce(G.ultimate_parent_issuer_lei,U.parent_lei) as lei, U.parent_name
      from rmi_20211120.utility_information as U left join gleif_mdt.gleif_direct_issuer_ultimate_issuer as G on U.parent_lei=G.direct_issuer_lei
      group by coalesce(G.ultimate_parent_issuer_lei,U.parent_lei), U.parent_name) as A
     left join sec_dera.t_r as R on A.lei=R.lei
where A.lei is not null
      and (R.lei is null or year(R.fy)=2020)
order by A.parent_name
""").fetchall()

print(len(l))
display(l)

92


[('2NUNNB7D43COUIRE5295', 'AES Corp.', 'AES CORP', '2020-01-01', 9660000000.0),
 ('549300T12EZ1F6PWWU29', 'Alcoa Corp.', 'ALCOA CORP', '2020-01-01', 9286000000.0),
 ('549300K5VIUTJXQL7X75', 'Algonquin Power & Utilities Corp.', 'ALGONQUIN POWER & UTILITIES CORP.', '2020-01-01', 1677058000.0),
 ('549300NNLSIMY6Z8OT86', 'Allete, Inc.', 'ALLETE INC', '2020-01-01', 1169100000.0),
 ('5493009ML300G373MZ12', 'Alliant Energy', 'ALLIANT ENERGY CORP', '2020-01-01', 3416000000.0),
 ('XRZQ5S7HYJFPHJ78L959', 'Ameren Corp.', 'AMEREN CORP', '2020-01-01', 5794000000.0),
 ('1B4S6S7G0TW5EE83BO58', 'American Electric Power Co., Inc.', 'AMERICAN ELECTRIC POWER CO INC', '2020-01-01', 14918500000.0),
 ('529900L26LIS2V8PWM23', 'American States Water', 'AMERICAN STATES WATER CO', '2020-01-01', 488243000.0),
 ('2138004PVNAV1E9SZP35', 'Antin Infrastructure Partners', None, None, None),
 ('549300GW1DWMX7FLDM29', 'ArcLight Capital Partners LLC', None, None, None),
 ('549300OX0Q38NLSKPB49', 'Avangrid, Inc.', 'AVANG

In [17]:
stop!

SyntaxError: invalid syntax (<ipython-input-17-8c726f94684c>, line 1)

In [ ]:
engine.execute("select count (*) from (select distinct(lei) from sec_dera.sub)").fetchall()

In [ ]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue, cash -- , debt --, assets, market_cap
from sec_dera.fy_revenue_by_lei as R
     join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     -- join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     -- join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()

In [ ]:
qres = engine.execute("""
explain analyze
select R.name, R.lei, R.tname, R.fy, R.sic, revenue,
    -- cash,
    debt,
    -- assets,
    market_cap
from sec_dera.fy_revenue_by_lei as R
     -- join sec_dera.cash_by_lei as C on R.name=C.name and R.fy=C.fy
     join sec_dera.debt_by_lei as D on R.name=D.name and R.fy=D.fy
     -- join sec_dera.assets_by_lei as A on R.name=A.name and R.fy=A.fy
     join sec_dera.float_by_lei as F on R.name=F.name and R.fy=F.fy
""")
l = qres.fetchall()